# Music recommender system

A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

Which type of recommender can we have?   

There are two main types of recommender systems: 
- Content-based filters
- Collaborative filters
  
> Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

### 1) Content-based filters

Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is **keyword matching**.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.

*How is this performed?*

In our case, because we are working with text and words, **Term Frequency-Inverse Document Frequency (TF-IDF)** can be used for this matching process.
  
We'll go through the steps for generating a **content-based** music recommender system.

### Importing required libraries

First, we'll import all the required libraries.

In [43]:
import numpy as np
import pandas as pd

In [44]:
from typing import List, Dict

We are going to use TfidfVectorizer from the Scikit-learn package.

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset


This dataset contains name, artist, and lyrics for *57650 songs in English*.

In [46]:
songs = pd.read_csv('songdata.csv')

In [47]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [48]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [49]:
songs['text'] = songs['text'].str.replace(r'\n', '')

/tmp/ipykernel_88038/269676193.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word.

In [50]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [51]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [52]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [53]:
similarities = {}

In [54]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [41]:
i=0
for k,v in similarities.items():
    if(i==10):
        break
    print(k,v)
    i+=1

Angels Unaware [(0.3223239634613378, 'Maybe Today', 'Bob Seger'), (0.2604143900434499, 'Lead Us Up The Mountain', 'Matt Redman'), (0.2561472613051227, 'Maybe I Could Change', 'Utopia'), (0.2378424117635568, 'Lead The Way', 'Mariah Carey'), (0.23176681863053814, "I'm Going Left", 'Eric Clapton'), (0.20130663744461746, 'Maybe In The End', 'Kenny Rogers'), (0.2004323556736315, 'Angels', 'Amy Grant'), (0.1991741929074009, 'Love Comes Tumbling', 'U2'), (0.18617124142480704, 'Highway', 'Moody Blues'), (0.17001027175458117, 'Wings Of A Lie', 'Starship'), (0.16865597035160734, 'Heartbreak Down', 'P!nk'), (0.1671381822075234, 'Girl', 'Tori Amos'), (0.16136479061875664, 'Maybe Tomorrow - The Jackson 5', 'Michael Jackson'), (0.15476841969200839, 'Maybe', 'Alison Krauss'), (0.15224585093546084, 'Prelude 12', 'Styx'), (0.14785954488460656, 'The Eye Of The Storm', 'Kris Kristofferson'), (0.14771736962723617, 'Between Heaven And Here', 'Kris Kristofferson'), (0.1394423475868869, 'A Means To An End', 

We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [55]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [56]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [58]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 6
}

In [59]:
recommedations.recommend(recommendation)

The 6 recommended songs for Manowar are:
Number 1:
Get Up Stand Up by Peter Tosh with 0.253 similarity score
--------------------
Number 2:
Forever by Omd with 0.217 similarity score
--------------------
Number 3:
Don't Fight It by Kenny Loggins with 0.19 similarity score
--------------------
Number 4:
King by Manowar with 0.178 similarity score
--------------------
Number 5:
I Think I Could Stand You Forever by Toto with 0.178 similarity score
--------------------
Number 6:
Stand My Ground by Within Temptation with 0.171 similarity score
--------------------


And we can pick another random song and recommend again:

In [60]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 5
}

In [61]:
recommedations.recommend(recommendation2)

The 5 recommended songs for I've Been Everywhere are:
Number 1:
It's A Man's World by Cher with 0.347 similarity score
--------------------
Number 2:
My Brother, My Enemy by Lucky Dube with 0.321 similarity score
--------------------
Number 3:
For You by Lana Del Rey with 0.31 similarity score
--------------------
Number 4:
I've Been Around by Hank Williams Jr. with 0.272 similarity score
--------------------
Number 5:
Man On The Silver Mountain by Rainbow with 0.266 similarity score
--------------------
